<a href="https://colab.research.google.com/github/kh9yaz/124Rus/blob/master/Task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Методы оптимизации

## Основы оптимизации

В этой работе мы рассмотрим основы оптимизации, для чего она используется, и рассмотрим базовые подходы,а так же познакомимся с возможностями Jupyter Notebook и Google Colab.

## Задача

Рассмотрим простую задачу для оптимизации. Начем с примера, рассмотренного в [статье](https://habr.com/ru/post/517056/).

Необходимо минимизировать функцию

$$
f(x) = 1.3 \cdot (x - y)^2 + 0.7 \cdot (x + y)^2
$$

используя [метод случайного поиска с вовзратом при неудачном шаге](https://studfile.net/preview/2152158/page:10/)

## Внимание!
Как [выяснилось](https://colab.research.google.com/drive/14uWejf0v1DII-CfcY48bl06i2zlSx-hl), интерактивность не поддерживается в Google Colab

In [ ]:
from IPython.display import HTML
# подключаем модули Jupyter Notebook для взаимодействия с пользователем
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

# пдключаем модули для расчетов и рисования графиков
import numpy as np
import matplotlib.pyplot as plt


# Служебная функция
# Возврашает элемент массива, переданного ей в качестве первого параметра с указанным номером
def step_slice(lst, step):
    return lst[step]


# Функция анимации. Именно она будет рисовать последовательность шагов
def animate_list(lst, play=False, interval=200):
    slider = widgets.IntSlider(min=0, max=len(lst) - 1, step=1, value=0)
    if play:
        play_widjet = widgets.Play(interval=interval)
        widgets.jslink((play_widjet, 'value'), (slider, 'value'))
        display(play_widjet)
    return interact(step_slice, lst=fixed(lst), step=slider)


# Отрисовка траектории и линий уровня функции
def plot_trajectory(func, traj, limit_point=None):
    # Добавим график, в ктором будем рисовать
    fig = plt.figure(figsize=(7, 7))
    ax = fig.add_axes([0, 0, 1, 1])

    # Если задаа финальная точка, обозначим ее зеленым кружком
    if limit_point:
        ax.plot([limit_point[0]], [limit_point[1]], 'o', color='green')

    delta = 0.025
    x = np.arange(-2, 2, delta)  # диапазон значений по x
    y = np.arange(-2, 2, delta)  # и по y соответственно
    X, Y = np.meshgrid(x, y)  # пространственная сетка по указанным диапазонам значений

    # вычисление значений функции во всех узлах сетки, необходимо для рисования изолиний на графике
    Z = np.zeros_like(X)
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Z[i][j] = func(X[i][j], Y[i][j])
    # создание контуров изолиний по заданным значениям функции
    CS = ax.contour(X, Y, Z, [0.5, 1.5, 3], colors=['blue', 'purple', 'red'])  # значения функции, для которых будут нарисованы изолинии, и соответствующие им цвета
    ax.plot([u[0] for u in traj], [u[1] for u in traj], color='black')  # рисование всех линий траектории
    ax.plot([u[0] for u in traj], [u[1] for u in traj], 'o', color='black')  # и соответствующих им точек

    plt.close(fig)
    return fig

Опишем функцию, которую хотим минимизировать, и собственно алгоритм минимизации

In [ ]:
display(HTML('<link rel="stylesheet" href="//stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css"/>'))
# Функция, которую минимизируем, минимум в точке (0, 0)
def f(x, y):
    return 1.3 * (x - y) ** 2 + 0.7 * (x + y) ** 2

x, y = (1.0, 1.0)  # начальная точка
num_iters = 50  # число шагов алгоритма (не факт, что минимум будет достигнут)
trajectory = [(x, y)]  # траектория пока состоит только из начальной точки
plots = []  # шаги алгоритма, которые будут анимированы, пока их нет

# Собственно алгоритм
for i in range(num_iters):  # выполняем num_iters шагов
    angle = 2 * np.pi * np.random.rand(1)  # случайным образом выбираем направленние движения
    dx, dy = (np.cos(angle)[0] / 2 / (i + 1) ** 0.5, np.sin(angle)[0] / 2 / (i + 1) ** 0.5)  # находим смещение

    trajectory.append((x + dx, y + dy))  # добавляем очередную найденую точку к таектории
    plots.append(plot_trajectory(f, trajectory, limit_point=(0, 0)))  # добавляем шаг работы алгоритма, который надо нарисовать в анимации

    if f(x, y) > f(x + dx, y + dy):  # если в выбранной точке значение целевой функции меньше, чем в предыдущей...
        x = x + dx  # перемещаемся
        y = y + dy  # в нее
    else:
        trajectory = trajectory[:-1]  # иначе вернемся на шаг назад

animate_list(plots, play=True, interval=300);  # выводим все наши шаги в виде анимации

Play(value=0, interval=300)

interactive(children=(IntSlider(value=0, description='step', max=49), Output()), _dom_classes=('widget-interac…

Вам необходимо выполнить следующие задачи:

1. Изменить алгоритм таким образом, чтобы останов алгоритма происходил:
    * при достижении заданной точности, или
    * по выполнению заданного числа шагов (в приведенной выше программе использовано только это условие)
2. Реализовать [алгоритм покоординатного спуска](http://www.machinelearning.ru/wiki/index.php?title=Метод_покоординатного_спуска) и анимацию его работы для той же функции

Для вашего удобства код алгоритма продублирован без изменений ниже для каждого из пунктов заданий. Вам необходимо модифицировать его в соответствии с указанным заданием.

1. Изменить алгоритм таким образом, чтобы останов алгоритма происходил при достижении заданной точности, или по выполнению заданного числа шагов

In [30]:
x, y = (1.0, 1.0)  # начальная точка
num_iters = 50  # число шагов алгоритма (не факт, что минимум будет достигнут)
trajectory = [(x, y)]  # траектория пока состоит только из начальной точки
plots = []  # шаги алгоритма, которые будут анимированы, пока их нет
number_of_steps = 45
precision = 0.0015

# Собственно алгоритм
for i in range(num_iters):  # выполняем num_iters шагов
    angle = 2 * np.pi * np.random.rand(1)  # случайным образом выбираем направленние движения
    dx, dy = (np.cos(angle)[0] / 2 / (i + 1) ** 0.5, np.sin(angle)[0] / 2 / (i + 1) ** 0.5)  # находим смещение

    trajectory.append((x + dx, y + dy))  # добавляем очередную найденую точку к таектории
    plots.append(plot_trajectory(f, trajectory, limit_point=(0, 0)))  # добавляем шаг работы алгоритма, который надо нарисовать в анимации

    if f(x, y) > f(x + dx, y + dy):  # если в выбранной точке значение целевой функции меньше, чем в предыдущей...
        x = x + dx  # перемещаемся
        y = y + dy  # в нее
    else:
        trajectory = trajectory[:-1]  # иначе вернемся на шаг назад

    if abs(f(x, y) - f(x + dx, y + dy)) <= precision:
     print ('Необходимая точность достигнута')
     break

    if i >= number_of_steps:
      print('Достигнут лимит количества шагов')
      break



animate_list(plots, play=True, interval=300);  # выводим все наши шаги в виде анимации

Достигнут лимит количества шагов


Play(value=0, interval=300)

interactive(children=(IntSlider(value=0, description='step', max=45), Output()), _dom_classes=('widget-interac…

2. Реализовать [алгоритм покоординатного спуска](http://www.machinelearning.ru/wiki/index.php?title=Метод_покоординатного_спуска) и анимацию его работы

In [29]:
x, y = (1.0, 1.0)  # начальная точка
num_iters = 50  # число шагов алгоритма (не факт, что минимум будет достигнут)
trajectory = [(x, y)]  # траектория пока состоит только из начальной точки
plots = []  # шаги алгоритма, которые будут анимированы, пока их нет

# Собственно алгоритм
for i in range(num_iters):  # выполняем num_iters шагов
    opt_var = 0
    angle = 2 * np.pi * np.random.rand(1)  # случайным образом выбираем направленние движения
    dx, dy = (np.cos(angle)[0] / 2 / (i + 1) ** 0.5, np.sin(angle)[0] / 2 / (i + 1) ** 0.5)  # находим смещение

    if opt_var == 0:
      new_step = (x+ dx, y)
      trajectory.append(new_step)
    elif opt_var == 1:
      new_step = (x, y + dy)
      trajectory.append(new_step)

    plots.append(plot_trajectory(f, trajectory, limit_point=(0, 0)))  # добавляем шаг работы алгоритма, который надо нарисовать в анимации

    if f(x, y) > f(new_step[0], new_step[1]):  # если в выбранной точке значение целевой функции меньше, чем в предыдущей...
        x = x + dx  # перемещаемся
        y = y + dy  # в нее
    else:
        trajectory = trajectory[:-1]  # иначе вернемся на шаг назад
        opt_var = 1 - opt_var

animate_list(plots, play=True, interval=300);  # выводим все наши шаги в виде анимации

Play(value=0, interval=300)

interactive(children=(IntSlider(value=0, description='step', max=49), Output()), _dom_classes=('widget-interac…